In [31]:
import Bio.Seq
import random
import logging
import itertools
import os
import re
import shutil
import tempfile
import subprocess
import numpy as np

from tral.paths import DATA_DIR, EXEC_DIR

LOG = logging.getLogger(__name__)

In [32]:
DATA_DIR

'/home/lina/.tral/data'

In [33]:
#sequence_type = 'DNA'
sequence_type = 'AA'

In [34]:
equilibrium_frequencies='human'
file = os.path.join(DATA_DIR, 'Random', "_".join(
                [sequence_type, equilibrium_frequencies, '3']) + '.txt')
print(file)

/home/lina/.tral/data/Random/AA_human_3.txt


In [7]:
with open(file, 'r') as f:
    a = f.readline()[:-1] # the [:-n] slices the \n in the very end

In [ ]:
#a = 'AAA 27355  AAR 13526  AAN 5245'

In [8]:
b = [i.split(' ') for i in a.split('  ')]

In [ ]:
#b = [tuple(i.split(' ')) for i in a.split('  ')]
# could be done as tuple but not necessary

In [ ]:
#a.split('  ')

In [9]:
b

[['ACC', '1234961'],
 ['TAG', '1418129'],
 ['ACA', '2110825'],
 ['AAA', '3730381'],
 ['ATC', '1416077'],
 ['AAC', '1565899'],
 ['ATA', '2150297'],
 ['AGG', '2036293'],
 ['CCT', '2038987'],
 ['ACT', '1765957'],
 ['AGC', '1578913'],
 ['AAG', '2319992'],
 ['AGA', '2469764'],
 ['CAT', '1990437'],
 ['AAT', '2690682'],
 ['ATT', '2697364'],
 ['CTG', '2295041'],
 ['CTA', '1416668'],
 ['CTC', '1833177'],
 ['CAC', '1539479'],
 ['ACG', '274180'],
 ['CAA', '1974628'],
 ['AGT', '1766247'],
 ['CCA', '1970632'],
 ['CCG', '312550'],
 ['CCC', '1603710'],
 ['TAT', '2153460'],
 ['GGT', '1238326'],
 ['TGT', '2125245'],
 ['CGA', '224729'],
 ['CAG', '2287773'],
 ['CGC', '253492'],
 ['GAT', '1419183'],
 ['CGG', '313198'],
 ['CTT', '2327283'],
 ['TGC', '1588311'],
 ['GGG', '1605495'],
 ['GGA', '1779124'],
 ['TAA', '2326091'],
 ['GGC', '1305435'],
 ['TAC', '1209580'],
 ['TTC', '2278442'],
 ['TCG', '225261'],
 ['TTA', '2329234'],
 ['TTT', '3745135'],
 ['GAC', '1077061'],
 ['TCC', '1778916'],
 ['GAA', '2273427']

In [ ]:
frequencies = {i[0]: int(i[1]) for i in b}

In [ ]:
frequencies

In [ ]:
alphabet = np.unique(i[0] for i in frequencies.keys())

# np.unique: find the unique elements of an array


In [ ]:
np.unique(i[0] for i in frequencies.keys())

In [ ]:
alphabet = np.unique([i[0] for i in frequencies.keys()])

In [ ]:
alphabet

In [13]:
def random_sequence(n_samples, sequence_type='AA', return_type='repeat',
                    equilibrium_frequencies='human', l=0, n=0,
                    sequence_length=0):
    """ Simulate random sequence locally.

    Simulate random sequence locally.

    Args:
        n_samples (int):  The number of samples
        sequence_type (str): Either "AA" or "DNA"
        return_type (str): Either "repeat" or "list"
        equilibrium_frequencies (str): Only "human" option available at current
        l (int): The length of the repeat unit
        n (int): The number of repeat units in the tandem repeat
        sequence_length (int): The total length of the simulated sequence

    Returns:
        Return type depends on ``return_type``.
    """

    if sequence_length == 0 and (l == 0 or n == 0):
        LOG.error('The specified sequence_length or the product of l and n was'
                  ' set to 0 for random_sequence simulation')
    else:
        if sequence_length == 0:
            sequence_length = l * n

        if equilibrium_frequencies == 'human':
            file = os.path.join(DATA_DIR, 'Random', "_".join(
                [sequence_type, equilibrium_frequencies, '3']) + '.txt')

        with open(file, 'r') as f:
            a = f.readline()[:-1]
        b = [i.split(' ') for i in a.split('  ')]
        frequencies = {i[0]: int(i[1]) for i in b}
        alphabet = np.unique([i[0] for i in frequencies.keys()]) #inclusion of []

        for _ in range(n_samples):
            seed_int = random.randint(1, sum(frequencies.values()))
            for key, value in frequencies.items():
                seed_int -= value
                if seed_int <= 0:
                    seed = key
                    break

            dimer = [''.join(i) for i in itertools.product(alphabet, repeat=2)]
            third_letter_frequencies = {
                iD: {
                    iA: frequencies[
                        iD +
                        iA] for iA in alphabet} for iD in dimer}

            sequence = seed
            for _ in range(sequence_length - 3):
                next_int = random.randint(
                    1, sum(third_letter_frequencies[sequence[-2:]].values()))
                for key, value in third_letter_frequencies[
                        sequence[-2:]].items():
                    next_int -= value
                    if next_int <= 0:
                        sequence += key
                        break

            # return Repeat instances
            if return_type == 'repeat' and not l == 0 and not n == 0:
                yield [sequence[i * l:(i + 1) * l] for i in range(n)], sequence_type
            elif return_type == 'list':
                yield sequence
            else:  # return seqIO instances
                yield Bio.Seq.Seq(sequence, sequence_type)


In [ ]:
random.randint(1, sum(frequencies.values()))

In [ ]:
#only for one sample
seed_int = random.randint(1, sum(frequencies.values()))
for key, value in frequencies.items():
    seed_int -= value
    print("seed_int",seed_int)
    print("seed",seed)
    if seed_int <= 0:
        print(seed)
        seed = key
        break


In [ ]:
#product('ABCD', repeat=2)
#AA AB AC AD BA BB BC BD CA CB CC CD DA DB DC DD
dimer = [''.join(i) for i in itertools.product(alphabet, repeat=2)]

In [ ]:
dimer

In [ ]:
x = itertools.product(alphabet, repeat=2)

In [ ]:
alphabet

In [ ]:
third_letter_frequencies = {
    iD: {
        iA: frequencies[
            iD +
            iA] for iA in alphabet} for iD in dimer}

In [ ]:
print(seed_int)

In [14]:
test_seq_list = random_sequence(2, sequence_type='DNA', return_type='list',equilibrium_frequencies='human', l=100, n=1)

In [ ]:
test_seq_repeat = random_sequence(3, sequence_type='AA', return_type='repeat',equilibrium_frequencies='human', l=1000, n=1)

In [ ]:
test_seq_else = random_sequence(3, sequence_type='AA', return_type='else',equilibrium_frequencies='human', l=10, n=5)

In [ ]:
print(test_seq_else) # gives Bio.Seq.Seq(sequence, sequence_type)
for i in test_seq_list:
    print("i",i)

In [15]:
print(test_seq_list)
for i in test_seq_list:
    print("i",i)

<generator object random_sequence at 0x7f02cebadb10>
i AAAAGCTTAAACAGCTGCAAGTGGCCTGGCAAGGAGAAGAGCTTTTTTCAGGTTTACCAGCTTTTAGCCCAAGAGTGCAGAGAGCATCTCACTCTTTCAG
i ATTTTGGACCAACAAGTGACCACCATCCTCGCATCTGCTTTCGATTCAGTCAAAGCCTTAAATACATTATCCACAGCCGTTCGTAACAGTGTATGACGAA


In [ ]:
print(test_seq_repeat)
for i in test_seq_repeat:
    print(i)

In [ ]:
test_seq_list

In [ ]:
[print(i) for i in test_seq]

In [35]:
runfile_template = os.path.join(DATA_DIR, "ALF", "template.drw")
alf_exec = os.path.join(EXEC_DIR, "alfsim")
# create temporary directory
working_dir = tempfile.mkdtemp()
LOG.debug("evolvedTR: Created tempfile: %s", working_dir)

In [5]:
runfile_template 
# manually created directory /home/lina/.tral/data/ALF and put alf-params.drw into it
# named as template.drw

'/home/lina/.tral/data/ALF/template.drw'

In [8]:
alf_exec

'/usr/local/bin/alfsim'

In [14]:
# create working dir
if not os.path.isdir(working_dir):
    os.makedirs(working_dir)

# Copy template file and append job specific info
runfilename = "alf.drw"
shutil.copyfile(runfile_template, os.path.join(working_dir, runfilename))

'/tmp/tmp2cb8pzx6/alf.drw'

In [16]:
# if indel_rate_per site:

indel_rate_per_site = 3
mutation_rate = 0.1

# insertion rate per site and PAM. E.g. for PAM=40 expect
# aaGainRate*40 insertions per site.
with open(os.path.join(working_dir, runfilename), "a") as runfile:
    runfile.write(
        "aaGainRate := " + str(indel_rate_per_site / mutation_rate) + ";\n")
    runfile.write(
        "aaLossRate := " + str(indel_rate_per_site / mutation_rate) + ";\n")
    runfile.write("maxIndelLength := 50;\n")
    runfile.write("indelModel := 'ZIPF';\n")
    runfile.write("Z_c := 1.821;\n")
    runfile.write("DawgPlacement := true;\n")

In [23]:
job_id = "test_id_name"
n_samples = 2
n = 3
l = 8

with open(os.path.join(working_dir, runfilename), "a") as runfile:
    runfile.write("uuid := '" + job_id + "';\n")
    runfile.write("mname := '" + job_id + "';\n")
    runfile.write("protStart := " + str(n_samples) + ";\n")
    runfile.write("NSpecies := " + str(n) + ";\n")
    runfile.write("minGeneLength := " + str(l) + ";\n")
    runfile.write("mutRate := " + str(mutation_rate) + ";\n")
    runfile.write("wdir := '" + working_dir + "';\n")

tree_length = n * mutation_rate

In [26]:
# parameters concerning the species tree
#if tree == 'star':
# BDTree, ToLSample, Custom
with open(os.path.join(working_dir, runfilename), "a") as runfile:
    runfile.write("treeType := 'Custom':\n")
    runfile.write(
        "tree := MakeStarTree(BirthDeathTree(0.1,0.1," + str(n) + ",10),mutRate):\n")
    runfile.write("treeLength:= %d ;\n" % tree_length)

In [ ]:
# BDTree, ToLSample, Custom

#else
runfile.write("treeType := 'BDTree':\n")
# From the last discussion with Daniel, only the ration of birth to
# death rate matters, if we scale tree to match pam distance
runfile.write("birthRate := 0.01:\n")
runfile.write("deathRate := 0.01:\n")
# for BDTree: should resulting tree be ultrametric, e.g. all leaves
# have same distance to origin?
runfile.write("ultrametric := false:\n")
runfile.write("treeLength:= %d ;\n" % tree_length)

In [17]:
print(runfile)

<_io.TextIOWrapper name='/tmp/tmp2cb8pzx6/alf.drw' mode='a' encoding='UTF-8'>


In [62]:
test_TRs = evolved_tandem_repeats(10, 3, 1, "AA", job_id='test',
                           mutation_rate=50, tree='birthdeath',
                           indel_rate_per_site=False,
                           return_type='repeat')

In [66]:
test_TRs = evolved_tandem_repeats(10, 3, 2, "DNA", job_id='test',
                           mutation_rate=50, tree='birthdeath',
                           indel_rate_per_site=False,
                           return_type='repeat')

In [67]:
for i in test_TRs:
    print(i)

ALFSIM was not able to produce simulated sequence.


FileNotFoundError: [Errno 2] No such file or directory: '/tmp/tmpj5a0dwql/test/MSA/MSA_all_dna.phy'

In [50]:
def evolved_tandem_repeats(l, n, n_samples, sequence_type, job_id='job_id',
                           mutation_rate=50, tree='star',
                           indel_rate_per_site=False,
                           return_type='repeat'):
    """ Simulate evolved sequences with ALF.

    Simulate evolved sequences with ALF:
    Dalquen, D. A., Anisimova, M., Gonnet, G. H. & Dessimoz, C.
    ALF--a simulation framework for genome evolution. Molecular Biology and Evolution 29,
    1115–1123 (2012).

    Args:
        l (int): The length of the repeat unit
        n (int): The number of repeat units in the tandem repeat
        n_samples (int):  The number of samples
        sequence_type (str): Either "AA" or "DNA"
        job_id (str): A tag for files produces with ALF, and result files.
        mutation_rate (float): The mutation rate.
        tree (str): The type of tree, e.g. "star" or "birthdeath"
        indel_rate_per_site (int or False): The indel rate per site.
        return_type (str): Either "repeat" or "list"

        sequence_length (int): The total length of the simulated sequence

    Returns:
        Return type depends on ``return_type``: ``Repeat`` or ``Bio.Seq.Seq`` instance.
    """

    runfile_template = os.path.join(DATA_DIR, "ALF", "template.drw")
    alf_exec = os.path.join(EXEC_DIR, "alfsim")
    # create temporary directory
    working_dir = tempfile.mkdtemp()
    LOG.debug("evolvedTR: Created tempfile: %s", working_dir)

    # create working dir
    if not os.path.isdir(working_dir):
        os.makedirs(working_dir)

    # Copy template file and append job specific info
    runfilename = "alf.drw"
    shutil.copyfile(runfile_template, os.path.join(working_dir, runfilename))

    with open(os.path.join(working_dir, runfilename), "a") as runfile:
        if indel_rate_per_site:
            # insertion rate per site and PAM. E.g. for PAM=40 expect
            # aaGainRate*40 insertions per site.
            runfile.write(
                "aaGainRate := " + str(indel_rate_per_site / mutation_rate) + ";\n")
            runfile.write(
                "aaLossRate := " + str(indel_rate_per_site / mutation_rate) + ";\n")
            runfile.write("maxIndelLength := 50;\n")
            runfile.write("indelModel := 'ZIPF';\n")
            runfile.write("Z_c := 1.821;\n")
            runfile.write("DawgPlacement := true;\n")
        runfile.write("uuid := '" + job_id + "';\n")
        runfile.write("mname := '" + job_id + "';\n")
        runfile.write("protStart := " + str(n_samples) + ";\n")
        runfile.write("NSpecies := " + str(n) + ";\n")
        runfile.write("minGeneLength := " + str(l) + ";\n")
        runfile.write("mutRate := " + str(mutation_rate) + ";\n")
        runfile.write("wdir := '" + working_dir + "';\n")

        tree_length = n * mutation_rate
        # parameters concerning the species tree
        if tree == 'star':
            # BDTree, ToLSample, Custom
            runfile.write("treeType := 'Custom':\n")
            runfile.write(
                "tree := MakeStarTree(BirthDeathTree(0.1,0.1," + str(n) + ",10),mutRate):\n")
            runfile.write("treeLength:= %d ;\n" % tree_length)
        else:
            # BDTree, ToLSample, Custom
            runfile.write("treeType := 'BDTree':\n")
            # From the last discussion with Daniel, only the ration of birth to
            # death rate matters, if we scale tree to match pam distance
            runfile.write("birthRate := 0.01:\n")
            runfile.write("deathRate := 0.01:\n")
            # for BDTree: should resulting tree be ultrametric, e.g. all leaves
            # have same distance to origin?
            runfile.write("ultrametric := false:\n")
            runfile.write("treeLength:= %d ;\n" % tree_length)
            # DANIEL: Is a tree := missing?
        if tree not in {'star', 'birthdeath'}:
            LOG.warning(
                "evolvedTR: tree input %s not known, assuming birthdeath tree",
                tree)

        # parameters concerning the substitution models
        if sequence_type == 'AA':
            runfile.write(
                "substModels := [SubstitutionModel('CustomP', ['" +
                os.path.join(
                    DATA_DIR,
                    'ALF',
                    'lg.dat') +
                "'])];\n")
            # CHECK! DOES
            # substModels := [SubstitutionModel('LG')];
            # WORK?
        elif sequence_type == 'DNA':
            runfile.write(
                "substModels := [SubstitutionModel('TN93', [.3, .4, .7],[seq(0.25,4)], true)]:\n")
            # CHECK! DO THE EMPIRICAL PARAMETERS MAKE SENSE AT ALL?
            # compare to http://people.inf.ethz.ch/ddalquen/alf/ALF_manual.pdf
        else:  # CODONS
            runfile.write("substModels := [SubstitutionModel('CPAM')];\n")

    # Determine ALF MSA output file path
    if sequence_type == 'AA':
        infile = os.path.join(working_dir, job_id, "MSA", "MSA_all_aa.phy")
    elif sequence_type == 'DNA':
        infile = os.path.join(working_dir, job_id, "MSA", "MSA_all_dna.phy")
    else:  # CODONS
        infile = os.path.join(
            working_dir,
            job_id,
            "MSA",
            "MSA_all_codon.phy")

    for i in range(10):
        with open(os.path.join(working_dir, "out.txt"), "w") as outfile:
            alf_process = subprocess.Popen([alf_exec,
                                            runfilename],
                                           cwd=working_dir,
                                           stdout=outfile,
                                           stderr=outfile,
                                           close_fds=True)
            alf_process.wait()
        if os.path.isfile(infile):
            break
    else:
        LOG.error('ALFSIM was not able to produce simulated sequence.')

    # shutil.rmtree('/cluster/home/infk/eschaper/spielwiese/')
    #shutil.copytree(working_dir, '/cluster/home/infk/eschaper/spielwiese/')

    """ Read in MSA data from ALF
        The following is a short parser for ALFsim output files
        yielding MSAs as lists of strings,
        based on a special flavour of  Felstein stein MSA files """

    # find a repeat uni
    pattern_start = re.compile("\d+ \d+")
    pattern_seq = re.compile("\S+[ ]+([A-Z\-]+)")

    # Our possible parser states:
    #
    # state 1: Find beginning of MSA (Felsenstein)
    # state 2: Find all repeat units

    # protein ::=
    #    \d \d
    #    \S/\S    repeatunits
    #

    state = 1
    with open(infile, "r") as infile:
        for i, line in enumerate(infile):
            LOG.debug("Line %d: %s", i, line[0:-1])

            if 1 == state:  # Find first repeat unit & save begin
                search = pattern_start.search(line)
                if search:
                    LOG.debug(" *(1->2) Found MSA start")
                    state = 2
                    msa = []

            elif 2 == state:  # Find all repeat units
                search = pattern_seq.search(line)
                if search:
                    LOG.debug(" *(2->2) Found another repeat unit")
                    msa.append(search.group(1))
                else:
                    LOG.debug(" *(2->1) repeat region finished, yielding.")
                    state = 1
                    # YIELD IF WE HAVE FOUND AT LEAST TWO REPEAT UNITS:
                    if len(msa) > 1:
                        if return_type == 'repeat':
                            yield repeat.Repeat(begin=0, msa=msa, sequence_type=sequence_type)
                        elif return_type == 'list':
                            yield msa
                        else:
                            LOG.debug("YIELD: %s",
                                      "".join(msa).replace('-', ''))
                            yield Bio.Seq.Seq("".join(msa).replace('-', ''), sequence_type)

    # delete temporary directory
    try:
        shutil.rmtree(working_dir)
    except OSError:
        pass


In [ ]:
# test directly in bash:
# alfsim /tmp/tmp0h0w116y/alf.drw stdout=/tmp/tmp0h0w116y/out.txt stderr=/tmp/tmp0h0w116y/out.txt close_fds=True

In [39]:
alf_exec # = alfsim

'/usr/local/bin/alfsim'

In [48]:
runfile = os.path.join(working_dir,runfilename)
outfile = os.path.join(working_dir, "out.txt") 
infile = os.path.join(working_dir, job_id, "MSA", "MSA_all_aa.phy") 

In [49]:
print(runfile)
print(outfile)
print(infile)

/tmp/tmp0h0w116y/alf.drw
/tmp/tmp0h0w116y/test_id_name/MSA/MSA_all_aa.phy
/tmp/tmp0h0w116y/out.txt


In [ ]:
def read_fasta(seq_filename, sequence_type='AA'):
    """ Read repeat from file in fasta format.

    Read repeat from file in fasta format.

    Args:
        seq_filename (str):  Path to the repeats containing fasta file
        sequence_type (str): Either "AA" or "DNA"

    Returns:
        (list of Repeat): A list of Repeat instances.
    """

    pat_start = re.compile(r">(.*)")
    pat_repeat_unit = re.compile(r"([\w\.\-]+)")

    # Our possible parser states:
    #
    # 1: searching for sequence name
    # 2: searching for repeat units

    repeats = {}
    state = 1
    with open(seq_filename, "rt") as infile:

        for i, line in enumerate(infile):
            LOG.debug("Line {0}: {1}".format(i, line[0:-1]))
            if 1 == state:
                match = pat_start.match(line)
                if match:
                    LOG.debug(" * (1->2) Found start")
                    LOG.debug("Start: %s", match.group(1))
                    name = match.group(1)
                    repeats[name] = []
                    state = 2

            elif 2 == state:
                match = pat_repeat_unit.match(line)
                if match:
                    LOG.debug(" * (2->2) Found Repeat unit")
                    LOG.debug("Repeat Unit: %s", match.group(1))
                    repeat_unit = match.group(1)
                    repeats[name].append(repeat_unit.replace(".", "-").upper())
                else:
                    LOG.debug(" * (2->1) Found NO further Repeat unit")
                    state = 1

    for i_name, iR in repeats.items():
        yield iR, sequence_type

In [ ]:
def save_repeat_fasta(tandem_repeats, file):
    """ save multiple <tandem_repeats> in Fasta format in specified <file>

        At current, only one TR per sequence can be defined, as the identifiers
        in the dict <tandem_repeats> must be unique.

        Parameters: Dict of tandem repeats and identifiers.
            e.g. {'ENSP00012': msa1, 'ENSP00013': msa2}

            >ID
            GHKI
            GHKI
            GH--
    """

    with open(file, 'w', newline='\n') as f:
        for identifier, msa in tandem_repeats.items():
            f.write(">{0}\n".format(identifier))
            f.write("\n".join(msa) + "\n\n")